This notebook includes the final model of the project, it has only been run on 2000 Hours which on average each have 7 observations so  ca 15.000 datapoints. We run out of memory at 8 GB machine, at more hours. Quick math shows that we should use nowhere near that much memory given the input and the model, so it is probably something to do with STAN, which we are not aware off.  The dataset can be downloaded with a kaggle account at https://www.kaggle.com/c/how-much-did-it-rain/data. 

In [1]:
import pandas as pd
import pystan
import numpy as np
import matplotlib.pyplot as plt
import pystan_utils
import scipy

In [38]:
Hours=1000
df=pd.read_csv("train_2013.csv",sep=",")
df=df.head(Hours+100)

The following code take the input from the data-set organizes it in a way that can be used by stan. Also it finds which datapoints are missing from the data, and the indices of theese to use for transformed parameters in stan. We exclude a hour of datapoints if the expected rainfall is over 100 mm, since this a super unlickly event and there is probably something wrong with the data. We also create the "Time" variable from the variables "TimeToEnd". The "TimeToEnd" variable is the number of minutes left in the hour for the observation. From the difference between the "TimeToEnd" of different observations from the same radar in the same hour, we create the "Time" variable as this difference. This way we assume that it always rains as much as the closest observation. For example given the observation from the same radar with "TimeToEnd" 58,50,30, we would create the corresponding "Time" variables as 10,20,30. As can be seen the first point also get the start period of the hour.

In [ ]:
Bad_Hours=0
Num_Obs = np.zeros(Hours).astype(int)
Expected = np.zeros(Hours)
N=0
i=0
count=0
while i-Bad_Hours<Hours:
    if df["Expected"][i]>100:
        Bad_Hours+=1
        i+=1
        continue
    N+=int(len(df["TimeToEnd"][i].split(' ')))
    Num_Obs[count]=int(len(df["TimeToEnd"][i].split(' ')))
    Expected[count]=df["Expected"][i]
    i+=1
    count+=1

The above code count the number of observations per hour, and exlcudes all the "Bad hours".

In [ ]:
HydrometeorType = np.zeros(N).astype(int)
TTE = np.zeros((Hours,int(max(Num_Obs))))
Time = np.zeros(N).astype(int)
Distance = np.zeros(N)
count=0
count2=0
for i in range(Hours+Bad_Hours):
    if df["Expected"][i]>100:
        continue
    Ht=[int(float(x)) for x in df["HydrometeorType"][i].split(' ')]
    T =[int(float(x)) for x in df["TimeToEnd"][i].split(' ')]
    D = [int(float(x)) for x in df["DistanceToRadar"][i].split(' ')]
    for j in range(len(Ht)):
        HydrometeorType[count]=int(Ht[j])
        TTE[count2,j]=int(T[j])
        Distance[count]=int(D[j])
        count+=1
    count2+=1

The above code takes out all the variables that cannot be missing from the pandas dataframe

In [ ]:
count=0
count2=0
for i in range(Hours+Bad_Hours):
    if df["Expected"][i]>100:
        continue
    Time[count]=60-TTE[count2,0]
    for j in range(int(Num_Obs[count2])):
        if j==int(Num_Obs[count2])-1:
            Time[count]=TTE[count2,j]
        elif TTE[count2,j+1]<=TTE[count2,j] and Distance[count]==Distance[count+1]:
            Time[count]+=(TTE[count2,j]-TTE[count2,j+1])
        else:
            Time[count]=TTE[count2,j]
            Time[count+1]=60-TTE[count2,j+1]
        count+=1
    count2+=1


The above codes creates the "Time" Variable from the "TimeToEnd" varaible. The below code is the main code part, that takes the rest of the data-variables out from the data frame. Count for each of them the amount which are missing and the indexes of these. Also we store all the indexes for the ones not missing. This is to be used in the Stan code for handeling missing data.

In [39]:
Composite=[]
HybridScan=[]
Reflectivity=[]
ReflectivityQC=[]
RhoHV=[]
Velocity=[]
Zdr=[]
RadarQualityIndex=[]
LogWaterVolume=[]
MassWeightedMean=[]
MassWeightedSD=[]
RR1=[]
RR2=[]
RR3=[]


Composite_Index=[]
HybridScan_Index=[]
Reflectivity_Index=[]
ReflectivityQC_Index=[]
RhoHV_Index=[]
Velocity_Index=[]
Zdr_Index=[]
RadarQualityIndex_Index=[]
LogWaterVolume_Index=[]
MassWeightedMean_Index=[]
MassWeightedSD_Index=[]
RR1_Index=[]
RR2_Index=[]
RR3_Index=[]

Composite_Missing_Index=[]
HybridScan_Missing_Index=[]
Reflectivity_Missing_Index=[]
ReflectivityQC_Missing_Index=[]
RhoHV_Missing_Index=[]
Velocity_Missing_Index=[]
Zdr_Missing_Index=[]
RadarQualityIndex_Missing_Index=[]
LogWaterVolume_Missing_Index=[]
MassWeightedMean_Missing_Index=[]
MassWeightedSD_Missing_Index=[]
RR1_Missing_Index=[]
RR2_Missing_Index=[]
RR3_Missing_Index=[]

Composite_Missing_Count=0
HybridScan_Missing_Count=0
Reflectivity_Missing_Count=0
ReflectivityQC_Missing_Count=0
RhoHV_Missing_Count=0
Velocity_Missing_Count=0
Zdr_Missing_Count=0
RadarQualityIndex_Missing_Count=0
LogWaterVolume_Missing_Count=0
MassWeightedMean_Missing_Count=0
MassWeightedSD_Missing_Count=0
RR1_Missing_Count=0
RR2_Missing_Count=0
RR3_Missing_Count=0

count=0
for i in range(Hours+Bad_Hours):
    if df["Expected"][i]>100:
        continue
    C=[float(x) for x in df["Composite"][i].split(' ')]
    HS=[float(x) for x in df["HybridScan"][i].split(' ')]
    R=[float(x) for x in df["Reflectivity"][i].split(' ')]
    RQC=[float(x) for x in df["ReflectivityQC"][i].split(' ')]
    RHV=[float(x) for x in df["RhoHV"][i].split(' ')]
    V=[float(x) for x in df["Velocity"][i].split(' ')]
    Z=[float(x) for x in df["Zdr"][i].split(' ')]
    RQI=[float(x) for x in df["RadarQualityIndex"][i].split(' ')]
    if(len(df["TimeToEnd"][i].split(' '))==1):
        tempLogWaterVolume=''+str(df["LogWaterVolume"][i])
        tempMassWeightedMean=''+str(df["MassWeightedMean"][i])
        tempMassWeightedSD=''+str(df["MassWeightedSD"][i])
        tempRR1=''+str(df["RR1"][i])
        tempRR2=''+str(df["RR2"][i])
        tempRR3=''+str(df["RR3"][i])
    else:
        tempLogWaterVolume=df["LogWaterVolume"][i]
        tempMassWeightedMean=df["MassWeightedMean"][i]
        tempMassWeightedSD=df["MassWeightedSD"][i]
        tempRR1=df["RR1"][i]
        tempRR2=df["RR2"][i]
        tempRR3=df["RR3"][i]
    LWV=[float(x) for x in tempLogWaterVolume.split(' ')]
    MWM=[float(x) for x in tempMassWeightedMean.split(' ')]
    MWSD=[float(x) for x in tempMassWeightedSD.split(' ')]
    R1=[float(x) for x in tempRR1.split(' ')]
    R2=[float(x) for x in tempRR2.split(' ')]
    R3=[float(x) for x in tempRR3.split(' ')]
    for j in range(len(C)):
        if C[j]==-99000.0 or C[j] ==-99901.0 or C[j]==-99903.0 or C[j]==-99900.0 or C[j]<-5 or C[j]>30:
            Composite_Missing_Index.append(count)
            Composite_Missing_Count+=1
        else:
            Composite_Index.append(count)
            Composite.append(C[j])
        if HS[j]==-99000.0 or HS[j] ==-99901.0 or HS[j]==-99903.0 or HS[j]==-99900.0 or HS[j]<-5 or HS[j]>30:
            HybridScan_Missing_Index.append(count)
            HybridScan_Missing_Count+=1
        else:
            HybridScan_Index.append(count)
            HybridScan.append(HS[j])
        if R[j]==-99000.0 or R[j] ==-99901.0 or R[j]==-99903.0 or R[j]==-99900.0 or R[j]<-30 or R[j]>55:
            Reflectivity_Missing_Index.append(count)
            Reflectivity_Missing_Count+=1
        else:
            Reflectivity_Index.append(count)
            Reflectivity.append(R[j])
        if RQC[j]==-99000.0 or RQC[j] ==-99901.0 or RQC[j]==-99903.0 or RQC[j]==-99900.0 or RQC[j]<-50 or RQC[j]>40:
            ReflectivityQC_Missing_Index.append(count)
            ReflectivityQC_Missing_Count+=1
        else:
            ReflectivityQC_Index.append(count)
            ReflectivityQC.append(RQC[j])
        if RHV[j]==-99000.0 or RHV[j] ==-99901.0 or RHV[j]==-99903.0 or RHV[j]==-99900.0 or RHV[j]<0 or RHV[j]>1:
            RhoHV_Missing_Index.append(count)
            RhoHV_Missing_Count+=1
        else:
            RhoHV_Index.append(count)
            RhoHV.append(RHV[j])
        if V[j]==-99000.0 or V[j] ==-99901.0 or V[j]==-99903.0 or V[j]==-99900.0 or V[j]<-25 or V[j]>50:
            Velocity_Missing_Index.append(count)
            Velocity_Missing_Count+=1
        else:
            Velocity_Index.append(count)
            Velocity.append(V[j])
        if Z[j]==-99000.0 or Z[j] ==-99901.0 or Z[j]==-99903.0 or Z[j]==-99900.0 or Z[j]>10 or Z[j]<-8:
            Zdr_Missing_Index.append(count)
            Zdr_Missing_Count+=1
        else:
            Zdr_Index.append(count)
            Zdr.append(Z[j])
        if RQI[j]==-99000.0 or RQI[j] ==-99901.0 or RQI[j]==-99903.0 or RQI[j]==-99900.0 or RQI[j]==999.0 or RQI[j]<0 or RQI[j]>1:
            RadarQualityIndex_Missing_Index.append(count)
            RadarQualityIndex_Missing_Count+=1
        else:
            RadarQualityIndex_Index.append(count)
            RadarQualityIndex.append(RQI[j])
        if LWV[j]==-99000.0 or LWV[j] ==-99901.0 or LWV[j]==-99903.0 or LWV[j]==-99900.0 or np.isnan(LWV[j]) or LWV[j]<-18 or LWV[j]>-2:
            LogWaterVolume_Missing_Index.append(count)
            LogWaterVolume_Missing_Count+=1
        else:
            LogWaterVolume_Index.append(count)
            LogWaterVolume.append(LWV[j])
        if MWM[j]==-99000.0 or MWM[j] ==-99901.0 or MWM[j]==-99903.0 or MWM[j]==-99900.0 or np.isnan(MWM[j]) or MWM[j]<1 or MWM[j]>4:
            MassWeightedMean_Missing_Index.append(count)
            MassWeightedMean_Missing_Count+=1
        else:
            MassWeightedMean_Index.append(count)
            MassWeightedMean.append(MWM[j])
        if MWSD[j]==-99000.0 or MWSD[j] ==-99901.0 or MWSD[j]==-99903.0 or MWSD[j]==-99900.0 or np.isnan(MWSD[j]) or MWSD[j]<0.3 or MWSD[j]>1.8:
            MassWeightedSD_Missing_Index.append(count)
            MassWeightedSD_Missing_Count+=1
        else:
            MassWeightedSD_Index.append(count)
            MassWeightedSD.append(MWSD[j])
        if R1[j]==-99000.0 or R1[j] ==-99901.0 or R1[j]==-99903.0 or R1[j]==-99900.0 or np.isnan(R1[j]) or R1[j]<0 or R1[j]>100:
            RR1_Missing_Index.append(count)
            RR1_Missing_Count+=1
        else:
            RR1.append(R1[j])
            RR1_Index.append(count)
        if R2[j]==-99000.0 or R2[j] ==-99901.0 or R2[j]==-99903.0 or R2[j]==-99900.0 or np.isnan(R2[j]) or R2[j]<0 or R2[j]>100:
            RR2_Missing_Index.append(count)
            RR2_Missing_Count+=1
        else:
            RR2.append(R2[j])
            RR2_Index.append(count)
        if R3[j]==-99000.0 or R3[j] ==-99901.0 or R3[j]==-99903.0 or R3[j]==-99900.0 or np.isnan(R3[j]) or R3[j]<0 or R3[j]>100:
            RR3_Missing_Index.append(count)
            RR3_Missing_Count+=1
        else:
            RR3.append(R3[j])
            RR3_Index.append(count)
        count+=1
Composite_Missing_Index=np.array(Composite_Missing_Index).astype(int)+1
HybridScan_Missing_Index=np.array(HybridScan_Missing_Index).astype(int)+1
Reflectivity_Missing_Index=np.array(Reflectivity_Missing_Index).astype(int)+1
ReflectivityQC_Missing_Index=np.array(ReflectivityQC_Missing_Index).astype(int)+1
RhoHV_Missing_Index=np.array(RhoHV_Missing_Index).astype(int)+1
Velocity_Missing_Index=np.array(Velocity_Missing_Index).astype(int)+1
Zdr_Missing_Index=np.array(Zdr_Missing_Index).astype(int)+1
RadarQualityIndex_Missing_Index=np.array(RadarQualityIndex_Missing_Index).astype(int)+1
LogWaterVolume_Missing_Index=np.array(LogWaterVolume_Missing_Index).astype(int)+1
MassWeightedMean_Missing_Index=np.array(MassWeightedMean_Missing_Index).astype(int)+1
MassWeightedSD_Missing_Index=np.array(MassWeightedSD_Missing_Index).astype(int)+1
RR1_Missing_Index=np.array(RR1_Missing_Index).astype(int)+1
RR2_Missing_Index=np.array(RR2_Missing_Index).astype(int)+1
RR3_Missing_Index=np.array(RR3_Missing_Index).astype(int)+1

Composite_Index=np.array(Composite_Index).astype(int)+1
HybridScan_Index=np.array(HybridScan_Index).astype(int)+1
Reflectivity_Index=np.array(Reflectivity_Index).astype(int)+1
ReflectivityQC_Index=np.array(ReflectivityQC_Index).astype(int)+1
RhoHV_Index=np.array(RhoHV_Index).astype(int)+1
Velocity_Index=np.array(Velocity_Index).astype(int)+1
Zdr_Index=np.array(Zdr_Index).astype(int)+1
RadarQualityIndex_Index=np.array(RadarQualityIndex_Index).astype(int)+1
LogWaterVolume_Index=np.array(LogWaterVolume_Index).astype(int)+1
MassWeightedMean_Index=np.array(MassWeightedMean_Index).astype(int)+1
MassWeightedSD_Index=np.array(MassWeightedSD_Index).astype(int)+1
RR1_Index=np.array(RR1_Index).astype(int)+1
RR2_Index=np.array(RR2_Index).astype(int)+1
RR3_Index=np.array(RR3_Index).astype(int)+1

Composite=np.array(Composite)
HybridScan=np.array(HybridScan)
Reflectivity=np.array(Reflectivity)
ReflectivityQC=np.array(ReflectivityQC)
RhoHV=np.array(RhoHV)
Velocity=np.array(Velocity)
Zdr=np.array(Zdr)
RadarQualityIndex=np.array(RadarQualityIndex)
LogWaterVolume=np.array(LogWaterVolume)
MassWeightedMean=np.array(MassWeightedMean)
MassWeightedSD=np.array(MassWeightedSD)
RR1=np.array(RR1)
RR2=np.array(RR2)
RR3=np.array(RR3)


The below cell show how many percent of the different input variables is missing, as we can see this ranges from 0 towards 90 %

In [40]:
print(Composite_Missing_Count/N,HybridScan_Missing_Count/N,Reflectivity_Missing_Count/N,ReflectivityQC_Missing_Count/N,RhoHV_Missing_Count/N,Velocity_Missing_Count/N,Zdr_Missing_Count/N,RadarQualityIndex_Missing_Count/N,LogWaterVolume_Missing_Count/N,MassWeightedMean_Missing_Count/N,MassWeightedSD_Missing_Count/N,RR1_Missing_Count/N,RR2_Missing_Count/N,RR3_Missing_Count/N)

0.6244959677419355 0.6941784274193549 0.0 0.6231098790322581 0.3146421370967742 0.3084677419354839 0.053805443548387094 0.19153225806451613 0.6993447580645161 0.6997227822580645 0.6993447580645161 0.011340725806451613 0.8566028225806451 0.9063760080645161


In [41]:
data = {'Hours': Hours, 'N': N,'Num_Obs': Num_Obs,'HydrometeorType':HydrometeorType,'Expected':Expected, 
        'Distance':Distance, 'Time':Time, 'Composite_Missing_Count':Composite_Missing_Count,
       'HybridScan_Missing_Count':HybridScan_Missing_Count, 'Reflectivity_Missing_Count':Reflectivity_Missing_Count,
        'ReflectivityQC_Missing_Count':ReflectivityQC_Missing_Count, 'RhoHV_Missing_Count':RhoHV_Missing_Count, 
        'Velocity_Missing_Count': Velocity_Missing_Count, 'Zdr_Missing_Count':Zdr_Missing_Count,
        'RadarQualityIndex_Missing_Count':RadarQualityIndex_Missing_Count, 'LogWaterVolume_Missing_Count':LogWaterVolume_Missing_Count, 
        'MassWeightedMean_Missing_Count':MassWeightedMean_Missing_Count, 'MassWeightedSD_Missing_Count':MassWeightedSD_Missing_Count, 
        'RR1_Missing_Count':RR1_Missing_Count, 'RR2_Missing_Count':RR2_Missing_Count, 'RR3_Missing_Count':RR3_Missing_Count,
        'Composite_Missing_Index':Composite_Missing_Index,
        'HybridScan_Missing_Index':HybridScan_Missing_Index, 'Reflectivity_Missing_Index':Reflectivity_Missing_Index,
        'ReflectivityQC_Missing_Index':ReflectivityQC_Missing_Index, 'RhoHV_Missing_Index':RhoHV_Missing_Index, 
        'Velocity_Missing_Index': Velocity_Missing_Index, 'Zdr_Missing_Index':Zdr_Missing_Index,
        'RadarQualityIndex_Missing_Index':RadarQualityIndex_Missing_Index, 'LogWaterVolume_Missing_Index':LogWaterVolume_Missing_Index, 
        'MassWeightedMean_Missing_Index':MassWeightedMean_Missing_Index, 'MassWeightedSD_Missing_Index':MassWeightedSD_Missing_Index, 
        'RR1_Missing_Index':RR1_Missing_Index, 'RR2_Missing_Index':RR2_Missing_Index, 'RR3_Missing_Index':RR3_Missing_Index,
        'Composite':Composite, 'HybridScan':HybridScan, 'Reflectivity':Reflectivity, 'ReflectivityQC':ReflectivityQC, 
        'RhoHV':RhoHV, 'Velocity':Velocity, 'Zdr':Zdr, 'RadarQualityIndex':RadarQualityIndex, 'LogWaterVolume':LogWaterVolume, 
        'MassWeightedMean':MassWeightedMean, 'MassWeightedSD':MassWeightedSD, 'RR1':RR1, 'RR2':RR2, 'RR3':RR3, 'Composite_Index':Composite_Index,
        'HybridScan_Index':HybridScan_Index, 'Reflectivity_Index':Reflectivity_Index,
        'ReflectivityQC_Index':ReflectivityQC_Index, 'RhoHV_Index':RhoHV_Index, 
        'Velocity_Index': Velocity_Index, 'Zdr_Index':Zdr_Index,
        'RadarQualityIndex_Index':RadarQualityIndex_Index, 'LogWaterVolume_Index':LogWaterVolume_Index, 
        'MassWeightedMean_Index':MassWeightedMean_Index, 'MassWeightedSD_Index':MassWeightedSD_Index, 
        'RR1_Index':RR1_Index, 'RR2_Index':RR2_Index, 'RR3_Index':RR3_Index
        
       }


The model defined below is Very Long. The cause of this is because 14 of the input variables can be missing. All of theese have different datapoints missing. Therefore we need 4 input variables for each input. The indicies they are missing/not missing. The number of missing, and the variable itself. In same way most of the parameters is to handle this missing data. In generel 90 % of everything in the code before model is to handle missing data. In the model we put prior on all the input data that can be missing. This priors have been made as described in the report and the previous milestones. The model itself is a "simple" linear model, of the expected rain. We weight the impact of the different observations based on the distance from the radar where the obsevation was made to the gauge, the Time variables for the observation and the quality of the measurement given by the variable RadarQualityIndex.

In [24]:
model_definition = """
data{
    int<lower=1> Hours; # Number of hours in the data set
    int<lower=1> N; # Number of Observations in the data set
    int<lower=1> Num_Obs[Hours]; # Number of observations for each hour
    int<lower=1,upper=15> HydrometeorType[N];
    real<lower=0,upper=100> Expected[Hours]; #The expected rainfall rate,i.e. what we try to predict
    real<lower=0> Distance[N];
    int<lower=0> Time[N];
    int<lower=0> Composite_Missing_Count;
    int<lower=0> HybridScan_Missing_Count;
    int<lower=0> Reflectivity_Missing_Count;
    int<lower=0> ReflectivityQC_Missing_Count;
    int<lower=0> RhoHV_Missing_Count;
    int<lower=0> Velocity_Missing_Count;
    int<lower=0> Zdr_Missing_Count;
    int<lower=0> RadarQualityIndex_Missing_Count;
    int<lower=0> LogWaterVolume_Missing_Count;
    int<lower=0> MassWeightedMean_Missing_Count;
    int<lower=0> MassWeightedSD_Missing_Count;
    int<lower=0> RR1_Missing_Count;
    int<lower=0> RR2_Missing_Count;
    int<lower=0> RR3_Missing_Count;
    int<lower=1> Composite_Missing_Index[Composite_Missing_Count];
    int<lower=1> HybridScan_Missing_Index[HybridScan_Missing_Count];
    int<lower=1> Reflectivity_Missing_Index[Reflectivity_Missing_Count];
    int<lower=1> ReflectivityQC_Missing_Index[ReflectivityQC_Missing_Count];
    int<lower=1> RhoHV_Missing_Index[RhoHV_Missing_Count];
    int<lower=1> Velocity_Missing_Index[Velocity_Missing_Count];
    int<lower=1> Zdr_Missing_Index[Zdr_Missing_Count];
    int<lower=1> RadarQualityIndex_Missing_Index[RadarQualityIndex_Missing_Count];
    int<lower=1> LogWaterVolume_Missing_Index[LogWaterVolume_Missing_Count];
    int<lower=1> MassWeightedMean_Missing_Index[MassWeightedMean_Missing_Count];
    int<lower=1> MassWeightedSD_Missing_Index[MassWeightedSD_Missing_Count];
    int<lower=1> RR1_Missing_Index[RR1_Missing_Count];
    int<lower=1> RR2_Missing_Index[RR2_Missing_Count];
    int<lower=1> RR3_Missing_Index[RR3_Missing_Count];
    int<lower=1> Composite_Index[N-Composite_Missing_Count];
    int<lower=1> HybridScan_Index[N-HybridScan_Missing_Count];
    int<lower=1> Reflectivity_Index[N-Reflectivity_Missing_Count];
    int<lower=1> ReflectivityQC_Index[N-ReflectivityQC_Missing_Count];
    int<lower=1> RhoHV_Index[N-RhoHV_Missing_Count];
    int<lower=1> Velocity_Index[N-Velocity_Missing_Count];
    int<lower=1> Zdr_Index[N-Zdr_Missing_Count];
    int<lower=1> RadarQualityIndex_Index[N-RadarQualityIndex_Missing_Count];
    int<lower=1> LogWaterVolume_Index[N-LogWaterVolume_Missing_Count];
    int<lower=1> MassWeightedMean_Index[N-MassWeightedMean_Missing_Count];
    int<lower=1> MassWeightedSD_Index[N-MassWeightedSD_Missing_Count];
    int<lower=1> RR1_Index[N-RR1_Missing_Count];
    int<lower=1> RR2_Index[N-RR2_Missing_Count];
    int<lower=1> RR3_Index[N-RR3_Missing_Count];
    real Composite[N-Composite_Missing_Count];
    real HybridScan[N-HybridScan_Missing_Count];
    real Reflectivity[N-Reflectivity_Missing_Count];
    real ReflectivityQC[N-ReflectivityQC_Missing_Count];
    real RhoHV[N-RhoHV_Missing_Count];
    real Velocity[N-Velocity_Missing_Count];
    real Zdr[N-Zdr_Missing_Count];
    real RadarQualityIndex[N-RadarQualityIndex_Missing_Count];
    real LogWaterVolume[N-LogWaterVolume_Missing_Count];
    real MassWeightedMean[N-MassWeightedMean_Missing_Count];
    real MassWeightedSD[N-MassWeightedSD_Missing_Count];
    real RR1[N-RR1_Missing_Count];
    real RR2[N-RR2_Missing_Count];
    real RR3[N-RR3_Missing_Count];
}
parameters{
    real<lower=-5, upper=30> Composite_Missing[Composite_Missing_Count];
    real<lower=-5, upper=30> HybridScan_Missing[HybridScan_Missing_Count];
    real<lower=-30, upper=55> Reflectivity_Missing[Reflectivity_Missing_Count];
    real<lower=-50, upper=40> ReflectivityQC_Missing[ReflectivityQC_Missing_Count];
    real<lower=0,upper=1> RhoHV_Missing[RhoHV_Missing_Count];
    real<lower=-25,upper=50> Velocity_Missing[Velocity_Missing_Count];
    real<lower=-10,upper=10> Zdr_Missing[Zdr_Missing_Count];
    real<lower=0,upper=1> RadarQualityIndex_Missing[RadarQualityIndex_Missing_Count];
    real<lower=-18,upper=-2> LogWaterVolume_Missing[LogWaterVolume_Missing_Count];
    real<lower=1,upper=4> MassWeightedMean_Missing[MassWeightedMean_Missing_Count];
    real<lower=0.3,upper=1.8> MassWeightedSD_Missing[MassWeightedSD_Missing_Count];
    real<lower=0,upper=100> RR1_Missing[RR1_Missing_Count];
    real<lower=0,upper=100> RR2_Missing[RR2_Missing_Count];
    real<lower=0,upper=100> RR3_Missing[RR3_Missing_Count];
    
    #Model parameters under here
    real<lower=0> sigma_reflec;
    real<lower=0> sigma_hybrid;
    real<lower=0> sigma_com;
    real<lower=0> sigma_rr2;
    real mass_a;
    real mass_b;
    real rr2_a;
    real rr2_b;
    real rr2_c;
    vector[13] mean_param;
    real<lower=0> var_param[2];
}
transformed parameters{
    real<lower=-5, upper=30> Composite_Total[N];
    real<lower=-5, upper=30> HybridScan_Total[N];
    real<lower=-30, upper=55> Reflectivity_Total[N];
    real<lower=-50, upper=40> ReflectivityQC_Total[N];
    real<lower=0,upper=1> RhoHV_Total[N];
    real<lower=-25,upper=50> Velocity_Total[N];
    real<lower=-10,upper=10> Zdr_Total[N];
    real<lower=0,upper=1> RadarQualityIndex_Total[N];
    real<lower=-18,upper=-2> LogWaterVolume_Total[N];
    real<lower=1,upper=4> MassWeightedMean_Total[N];
    real<lower=0.3,upper=1.8> MassWeightedSD_Total[N];
    real<lower=0,upper=100> RR1_Total[N];
    real<lower=0,upper=100> RR2_Total[N];
    real<lower=0,upper=100> RR3_Total[N];
    for(i in 1:Composite_Missing_Count){
        Composite_Total[Composite_Missing_Index[i]]=Composite_Missing[i];
    }
    for(i in 1:(N-Composite_Missing_Count)){
        Composite_Total[Composite_Index[i]]=Composite[i];
    }
    for(i in 1:HybridScan_Missing_Count){
        HybridScan_Total[HybridScan_Missing_Index[i]]=HybridScan_Missing[i];
    }
    for(i in 1:(N-HybridScan_Missing_Count)){
        HybridScan_Total[HybridScan_Index[i]]=HybridScan[i];
    }
    for(i in 1:Reflectivity_Missing_Count){
        Reflectivity_Total[Reflectivity_Missing_Index[i]]=Reflectivity_Missing[i];
    }
    for(i in 1:(N-Reflectivity_Missing_Count)){
        Reflectivity_Total[Reflectivity_Index[i]]=Reflectivity[i];
    }
    for(i in 1:ReflectivityQC_Missing_Count){
        ReflectivityQC_Total[ReflectivityQC_Missing_Index[i]]=ReflectivityQC_Missing[i];
    }
    for(i in 1:(N-ReflectivityQC_Missing_Count)){
        ReflectivityQC_Total[ReflectivityQC_Index[i]]=ReflectivityQC[i];
    }
    for(i in 1:RhoHV_Missing_Count){
        RhoHV_Total[RhoHV_Missing_Index[i]]=RhoHV_Missing[i];
    }
    for(i in 1:(N-RhoHV_Missing_Count)){
        RhoHV_Total[RhoHV_Index[i]]=RhoHV[i];
    }
    for(i in 1:Velocity_Missing_Count){
        Velocity_Total[Velocity_Missing_Index[i]]=Velocity_Missing[i];
    }
    for(i in 1:(N-Velocity_Missing_Count)){
        Velocity_Total[Velocity_Index[i]]=Velocity[i];
    }
    for(i in 1:Zdr_Missing_Count){
        Zdr_Total[Zdr_Missing_Index[i]]=Zdr_Missing[i];
    }
    for(i in 1:(N-Zdr_Missing_Count)){
        Zdr_Total[Zdr_Index[i]]=Zdr[i];
    }
    for(i in 1:RadarQualityIndex_Missing_Count){
        RadarQualityIndex_Total[RadarQualityIndex_Missing_Index[i]]=RadarQualityIndex_Missing[i];
    }
    for(i in 1:(N-RadarQualityIndex_Missing_Count)){
        RadarQualityIndex_Total[RadarQualityIndex_Index[i]]=RadarQualityIndex[i];
    }
    for(i in 1:LogWaterVolume_Missing_Count){
        LogWaterVolume_Total[LogWaterVolume_Missing_Index[i]]=LogWaterVolume_Missing[i];
    }
    for(i in 1:(N-LogWaterVolume_Missing_Count)){
        LogWaterVolume_Total[LogWaterVolume_Index[i]]=LogWaterVolume[i];
    }
    for(i in 1:MassWeightedMean_Missing_Count){
        MassWeightedMean_Total[MassWeightedMean_Missing_Index[i]]=MassWeightedMean_Missing[i];
    }
    for(i in 1:(N-MassWeightedMean_Missing_Count)){
        MassWeightedMean_Total[MassWeightedMean_Index[i]]=MassWeightedMean[i];
    }
    for(i in 1:MassWeightedSD_Missing_Count){
        MassWeightedSD_Total[MassWeightedSD_Missing_Index[i]]=MassWeightedSD_Missing[i];
    }
    for(i in 1:(N-MassWeightedSD_Missing_Count)){
        MassWeightedSD_Total[MassWeightedSD_Index[i]]=MassWeightedSD[i];
    }
    for(i in 1:RR1_Missing_Count){
        RR1_Total[RR1_Missing_Index[i]]=RR1_Missing[i];
    }
    for(i in 1:(N-RR1_Missing_Count)){
        RR1_Total[RR1_Index[i]]=RR1[i];
    }
    for(i in 1:RR2_Missing_Count){
        RR2_Total[RR2_Missing_Index[i]]=RR2_Missing[i];
    }
    for(i in 1:(N-RR2_Missing_Count)){
        RR2_Total[RR2_Index[i]]=RR2[i];
    }
    for(i in 1:RR3_Missing_Count){
        RR3_Total[RR3_Missing_Index[i]]=RR3_Missing[i];
    }
    for(i in 1:(N-RR3_Missing_Count)){
        RR3_Total[RR3_Index[i]]=RR3[i];
    }
}
model{
    real cancellationConstant=0.0001;
    int count=1;
    real totalD;
    real totalR;
    int totalT;
    real ExpectedMean;
    vector[13] tempData;
    vector[15] mu_rr1 = [0, 1.038, 29.0, 28.89, 0.338, 0.0, 0.0, 0.0, 0.0051, 0.743, 2.94, 0.064, 0.0, 8.356, 0.0]'; #mean of the different hydrometer classes
    vector[15] sigma_rr1 = [0.001, 2.45, 23.53, 38.714, 1.82, 0.001, 0.001, 0.001, 2.25, 3.253, 5.104, 0.932, 0.001, 11.13, 0.001]';#variance of the different hydrometer classes
    sigma_reflec ~ inv_gamma(4,2);
    sigma_hybrid ~ inv_gamma(1,2);
    sigma_com ~ inv_gamma(1,2);
    sigma_rr2 ~ inv_gamma(1,2);
    mass_a ~ normal(1.6,10);
    mass_b ~ normal(0.38,2);
    rr2_a ~ normal(0,1);
    rr2_b ~ normal(0,10);
    rr2_c ~ normal(0,10);
    mean_param ~ normal(0,10);
    var_param  ~ normal(0,10);
    for(i in 1:N){
        Reflectivity_Total[i] ~ normal(11.37,0.1328);
        Zdr_Total[i] ~ normal(1.41,8.23);
        ReflectivityQC_Total[i] ~ normal(Reflectivity_Total[i],sigma_reflec);
        HybridScan_Total[i] ~ normal(ReflectivityQC_Total[i],10*sigma_hybrid);
        MassWeightedSD_Total[i] ~ gamma(19,0.1);
        MassWeightedMean_Total[i] ~ normal(mass_a+mass_b*Zdr_Total[i],MassWeightedSD_Total[i]*MassWeightedSD_Total[i]);
        Composite_Total[i] ~ normal(ReflectivityQC_Total[i],10*sigma_com); # Should be half normal
        RR2_Total[i] ~ normal(rr2_a+rr2_b*ReflectivityQC_Total[i]+rr2_c*Zdr_Total[i],sigma_rr2);
        RR1_Total[i] ~ normal(mu_rr1[HydrometeorType[i]],sigma_rr1[HydrometeorType[i]]);
        RR3_Total[i] ~ gamma(1,10);
        #RadarQualityIndex_Total[i] ~ normal(0,5);
        target+=normal_lpdf(RadarQualityIndex_Total[i]|0,1)+normal_lpdf(RadarQualityIndex_Total[i]|1,1);
        RhoHV_Total[i] ~ normal(0.819,0.045);
        Velocity_Total[i] ~ normal(-0.26,5);
        LogWaterVolume_Total[i] ~ normal(-11.54,3.57);
    }
    for(i in 1:Hours){
        totalD=0;
        totalT=0;
        totalR=0;
        ExpectedMean=0;
        for(j in 1:Num_Obs[i]){
            totalD+=1/(Distance[count]+cancellationConstant);
            totalT+=Time[count];
            totalR+=RadarQualityIndex_Total[count];
            tempData[1]=Composite_Total[count];
            tempData[2]=HybridScan_Total[count];
            tempData[3]=Reflectivity_Total[count];
            tempData[4]=ReflectivityQC_Total[count];
            tempData[5]=RhoHV_Total[count];
            tempData[6]=Velocity_Total[count];
            tempData[7]=Zdr_Total[count];
            tempData[8]=LogWaterVolume_Total[count];
            tempData[9]=MassWeightedMean_Total[count];
            tempData[10]=MassWeightedSD_Total[count];
            tempData[11]=RR1_Total[count];
            tempData[12]=RR2_Total[count];
            tempData[13]=RR3_Total[count];
            ExpectedMean += (tempData'*mean_param)*Time[count]*RadarQualityIndex_Total[count]/(Distance[count]+cancellationConstant);
            count+=1;
        }
        
        
        ExpectedMean=ExpectedMean/(totalD*Num_Obs[i]*totalT*totalR+cancellationConstant);
        Expected[i] ~ normal(ExpectedMean,var_param[1]+(fabs(ExpectedMean)+cancellationConstant)*var_param[2]);
    }
}
generated quantities{
real cancellationConstant=0.0001;
vector[Hours] modelExpected;
vector[Hours] modelVariance;
vector[13] tempData;
real ExpectedMean;
real totalD;
real totalR;
int totalT;
int count=1;
for(i in 1:Hours){
    totalD=0;
    totalT=0;
    totalR=0;
    ExpectedMean=0;
    for(j in 1:Num_Obs[i]){
        totalD+=1/(Distance[count]+cancellationConstant);
        totalT+=Time[count];
        totalR+=RadarQualityIndex_Total[count];
        tempData[1]=Composite_Total[count];
        tempData[2]=HybridScan_Total[count];
        tempData[3]=Reflectivity_Total[count];
        tempData[4]=ReflectivityQC_Total[count];
        tempData[5]=RhoHV_Total[count];
        tempData[6]=Velocity_Total[count];
        tempData[7]=Zdr_Total[count];
        tempData[8]=LogWaterVolume_Total[count];
        tempData[9]=MassWeightedMean_Total[count];
        tempData[10]=MassWeightedSD_Total[count];
        tempData[11]=RR1_Total[count];
        tempData[12]=RR2_Total[count];
        tempData[13]=RR3_Total[count];
        ExpectedMean += (tempData'*mean_param)*Time[count]*RadarQualityIndex_Total[count]/(Distance[count]+cancellationConstant);
        count+=1;
    }


    ExpectedMean=ExpectedMean/(totalD*Num_Obs[i]*totalT*totalR+cancellationConstant);
    modelExpected[i]=ExpectedMean;
    modelVariance[i]=var_param[1]+(fabs(ExpectedMean)+cancellationConstant)*var_param[2];
}
}
"""

In [25]:
model = pystan.StanModel(model_code=model_definition)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_7d2531b6fc694f5f2846660c7c210451 NOW.
C:\Users\Johan\Anaconda3\lib\site-packages\Cython\Compiler\Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: C:\Users\Johan\AppData\Local\Temp\tmpqgsndsg9\stanfit4anon_model_7d2531b6fc694f5f2846660c7c210451_6593572120248076059.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [42]:
fit = model.vb(data=data, iter=5000, seed=42, verbose=True)

In [43]:
modelExpected=pystan_utils.vb_extract_variable(fit,"modelExpected", var_type="vector",dims=[Hours])
modelVariance=pystan_utils.vb_extract_variable(fit,"modelVariance", var_type="vector",dims=[Hours])

The function below is the evaluation function used for this problem as described on https://www.kaggle.com/c/how-much-did-it-rain/overview/evaluation. This error is based on a cumulative distribution function of the normal distribution of the predicted rainfall. 

In [44]:
def Heavyside_step_function(n,z):
    if n-z>=0:
        return 1
    else:
        return 0
def ErrorFunction(means,variances,reals):
    error=0
    N=len(reals)
    for i in range(N):
        mean=means[i]
        variance=variances[i]
        real=reals[i]
        for j in range(70):
            error+=((0.5*(1+scipy.special.erf((j-mean)/(np.sqrt(variance)*np.sqrt(2)))))-(Heavyside_step_function(i,real)))**2
    error=error/(70*N)
    return error

In [45]:
print(ErrorFunction(modelExpected,modelVariance,Expected))

0.005056108320222177


The parameters of the model below is the parmeters of the linear model. So which input variables has what influence on the guess of how much it rained that hour. 

In [47]:
meanparam=pystan_utils.vb_extract_variable(fit,"mean_param", var_type="vector",dims=[Hours])
varparam=pystan_utils.vb_extract_variable(fit,"var_param", var_type="vector",dims=[Hours])

In [48]:
print(meanparam,varparam)

[ 0.01803597  0.01654603 -0.0072688  -0.02256441  0.99241272 -0.03336553
  0.08080655  0.08130831  0.61908346 -0.5786402  -0.28270688  0.00363417
 -1.4410472 ] [1.92439255 0.57038588]
